In [1]:
import numpy as np
import pandas as pd

# Lab 9 - Multi-layer Perceptron Forward Pass & Backpropagation

## Part I
For this exercise you will implement a simple 2-layer perceptron with the forward pass and the backpropagation to learn the weights

For the first part you'll build and train a 2-layer neural network that predicts the prices of houses, using the usual Boston housing dataset.

In [2]:
boston = pd.read_csv('data/BostonHousing.txt')
boston.head(5)

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


As usual, consider the MEDV as your target variable. 
* Split the data into training, validation and testing (70,15,15)%
* Experiment with different number of neurons per layer for your network, using the validation set

In [3]:
# your code goes here
from sklearn.model_selection import train_test_split

X = boston.drop('medv', axis=1)
y = boston['medv']

features = X.shape[1]
predictions = 1

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.50, random_state=42)

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat
427,37.66190,0.0,18.10,0,0.679,6.202,78.7,1.8629,24,666,20.2,18.82,14.52
490,0.20746,0.0,27.74,0,0.609,5.093,98.0,1.8226,4,711,20.1,318.43,29.68
429,9.33889,0.0,18.10,0,0.679,6.380,95.6,1.9682,24,666,20.2,60.72,24.08
327,0.24103,0.0,7.38,0,0.493,6.083,43.7,5.4159,5,287,19.6,396.90,12.79
97,0.12083,0.0,2.89,0,0.445,8.069,76.0,3.4952,2,276,18.0,396.90,4.21


In [4]:
from sklearn.preprocessing import StandardScaler
X_train_arr = X_train.values
y_train_arr = y_train.values


scaler = StandardScaler()
X_train_normalized = np.array(scaler.fit_transform(X_train_arr))

X_train_arr_3d = X_train_arr.reshape(-1, 1, 13)
y_train_arr_3d = y_train_arr.reshape(-1, 1, 1)
X_train_normalized_3d = X_train_normalized.reshape(-1, 1, 13)

In [5]:
# activation function and its derivative
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_prime(x):
    return sigmoid(x) * (1 - sigmoid(x))

def identity(x):
    return x

def identity_prime(x):
    return 1

# loss function and its derivative
def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2))

def mse_prime(y_true, y_pred):
    return (y_pred - y_true)/y_true.size # can be used to mini-batch

In [6]:
class FCLayer():
    def __init__(self, input_size, output_size, activation, activation_prime):
        self.input = None
        self.output = None
        self.output_without_activation = None
        self.weights = np.random.rand(input_size, output_size) - 0.5
        self.bias = np.random.rand(1, output_size) - 0.5
        self.activation = activation
        self.activation_prime = activation_prime

    # forward propagation through the layer
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output_without_activation = np.dot(self.input, self.weights) + self.bias
        self.output = self.activation(self.output_without_activation)
        return self.output

    # computes dE/dW (weights), dE/dB (bias) for a given delta_k, from the subsequent layer. Returns delta_j, from this layer.
    def backward_propagation(self, output_error, learning_rate):
        delta_k_activated = self.activation_prime(self.output_without_activation) * output_error
        delta_j = np.dot(delta_k_activated, self.weights.T)
        gradiente = np.dot(self.input.T, delta_k_activated)

        # update parameters
        self.weights -= learning_rate * gradiente
        self.bias -= learning_rate * delta_k_activated # 1 * delta_k_activated, because input is 1 for bias
        return delta_j

In [7]:
class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None
        self.errors_during_train = []

    # add layer to network
    def add(self, layer):
        self.layers.append(layer)

    # set loss to use
    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime

    # predict output for given input
    def predict(self, input_data):
        samples = len(input_data)
        result = []

        # run network over all samples
        for i in range(samples):
            # forward propagation
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)

        return result

    # train the network
    def fit(self, x_train, y_train, epochs, learning_rate):
        # sample dimension first
        samples = len(x_train)

        # training loop
        for i in range(epochs):
            err = 0
            for j in range(samples):
                # forward propagation
                output = x_train[j]
                for layer in self.layers:
                    output = layer.forward_propagation(output)

                # compute loss (for display purpose only)
                err += self.loss(y_train[j], output)

                # backward propagation
                error = self.loss_prime(y_train[j], output)
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, learning_rate)

            # calculate average error on all samples
            err /= samples
            print('epoch %d/%d   error=%f' % (i+1, epochs, err))

In [15]:
# network
net = Network()
net.add(FCLayer(features, 15, sigmoid, sigmoid_prime))
net.add(FCLayer(15, predictions, identity, identity_prime))

# train
net.use(mse, mse_prime)
net.fit(X_train_normalized_3d, y_train_arr_3d, epochs=10000, learning_rate=0.08)

# test
out = net.predict(X_train_normalized_3d)
print(out)

epoch 1/10000   error=46.603175
epoch 2/10000   error=27.821730
epoch 3/10000   error=18.165562
epoch 4/10000   error=16.569899
epoch 5/10000   error=13.317981
epoch 6/10000   error=11.176216
epoch 7/10000   error=10.222435
epoch 8/10000   error=10.418387
epoch 9/10000   error=9.762582
epoch 10/10000   error=9.338196
epoch 11/10000   error=8.596385
epoch 12/10000   error=8.422019
epoch 13/10000   error=9.429062
epoch 14/10000   error=10.134335
epoch 15/10000   error=8.771449
epoch 16/10000   error=7.465984
epoch 17/10000   error=8.271563
epoch 18/10000   error=7.456159
epoch 19/10000   error=6.985249
epoch 20/10000   error=6.878858
epoch 21/10000   error=5.822730
epoch 22/10000   error=7.075134
epoch 23/10000   error=5.514007
epoch 24/10000   error=5.425992
epoch 25/10000   error=5.353077
epoch 26/10000   error=4.903993
epoch 27/10000   error=4.391010
epoch 28/10000   error=3.947481
epoch 29/10000   error=4.336443
epoch 30/10000   error=4.350539
epoch 31/10000   error=4.264262
epoch 32